## Importando dados

In [2]:
import pandas
import numpy

def read_labels():
    labels = []
    file = open('../data/features_names.txt', 'r')
    for line in file:
        line = line[:-1]
        labels.append(line)
    return labels

print(str(read_labels()))

['water_flow', 'temperature', 'internal_pressure', 'ph', 'gas_production', 'volume']


In [3]:
NUMBER_SAMPLES = 1000
NUMBER_FEATURES = 6

def create_mock_data():

    samples = numpy.around(1 * numpy.random.rand(NUMBER_SAMPLES, NUMBER_FEATURES), decimals=3)

    data = pandas.DataFrame(data=samples)
    print(str(data))
    data.to_csv('../data/biodigester.csv', index=False, encoding='utf-8')

In [7]:
import pandas

def read_data():
    labels = read_labels()
    biodigester_data = pandas.read_csv('../data/biodigester.csv', sep=",", names=labels)
    return biodigester_data

#print(str(read_data()))

In [8]:
bio_dataframe = read_data()
#print(bio_dataframe)

target = bio_dataframe['volume']
data = bio_dataframe.drop(['volume'], axis=1)

## Separando slice para validação

In [9]:
# particionar dados em treino e teste
from sklearn.cross_validation import train_test_split
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size = .1) # .5 metade dos dados para teste

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
print(len(data_train))
print(len(data_test))
print(len(target_train))
print(len(target_test))

900
100
900
100


## Gerando o modelo de regressão linear

In [61]:
from sklearn import linear_model
regressor = linear_model.LinearRegression()

# Train the model using the training sets
regressor.fit(data_train, target_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [13]:
X_toPredict = numpy.array([[0.817,0.065,0.774,0.72,0.104], [0.153,0.577,0.347,0.74,0.073]])

y_prediction = regressor.predict(X_toPredict)
print(y_prediction)

reg2 = regressor
y2 = reg2.predict(X_toPredict)
print(y2)

[0.49242922 0.47034935]
[0.49242922 0.47034935]


## Validando o modelo

In [14]:
from sklearn.cross_validation import cross_val_predict

predicted = cross_val_predict(regressor, data, target, cv=10)
print(y_pred.mean())

NameError: name 'y_pred' is not defined

In [ ]:
plt.scatter(data_test, target_test,  color='black')
plt.plot(data_test, predicted, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
import matplotlib.pyplot as plt
# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:

fig, ax = plt.subplots()
ax.scatter(target_test, predicted, edgecolors=(0, 0, 0))
ax.plot([target_test.min(), target_test.max()], [target_test.min(), target_test.max()], 'k--', lw=1)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

## Fazendo predições

In [17]:
X_toPredict = [[0.817,0.065,0.774,0.72,0.104], [0.153,0.577,0.347,0.74,0.073]]

y_prediction = regressor.predict(X_toPredict)
print(y_prediction)

[0.49242922 0.47034935]


## Fazendo testes

In [ ]:
X = data
y = target

# particionar dados em treino e teste
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .1) # .5 metade dos dados para teste

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

In [ ]:
import matplotlib.pyplot as plt
# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
predicted = cross_val_predict(regressor, data, target, cv=10)

fig, ax = plt.subplots()
ax.scatter(target, predicted, edgecolors=(0, 0, 0))
ax.plot([target.min(), target.max()], [target.min(), target.max()], 'k--', lw=1)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:

import matplotlib.pyplot as plt
# Plot outputs
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

plt.scatter(X_test, y_test,  color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()


## Persistencia do modelo

In [63]:
import pickle
from sklearn.externals import joblib

joblib.dump(regressor, 'modelo.pkl')

['modelo.pkl']

In [67]:
classificador = joblib.load('./modelo.pkl')
classificador.predict(X_toPredict)
#[0.49242922 0.47034935]

array([0.49242922, 0.47034935])

## Multi output

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn import linear_model

regressor = linear_model.LinearRegression()

regr_multirf = MultiOutputRegressor(regressor)
regr_multirf.fit(data_train, target_train)

In [50]:
data_2 = bio_dataframe.drop(['volume', 'gas_production'], axis=1)
target_2 = bio_dataframe[['volume', 'gas_production']]
#print(targets)
#print(data_2)

regressor = linear_model.LinearRegression()

regr_multirf = MultiOutputRegressor(regressor)
regr_multirf.fit(data_2, target_2)

MultiOutputRegressor(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
           n_jobs=1)

In [51]:
to = [[0.439, 0.394, 0.957, 0.182], [0.535, 0.049, 0.842, 0.894]]
print(regr_multirf.predict(to))

[[0.50654771 0.51124139]
 [0.48458505 0.47388229]]


In [68]:
from sklearn.metrics import r2_score

r = linear_model.LinearRegression()
r.fit(data_train, target_train)
y_pred = r.predict(data_test)
print(r2_score(target_test,y_pred))

-0.014345695111016132


In [1]:
print(pandas.DataFrame(data=y_pred, names=['bosta']))

NameError: name 'pandas' is not defined